In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Creating a P2WPKH transaction

In this section we'll create a P2WPKH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

## Reading
- Andreas Antonopoulos - Mastering Bitcoin Chapter 7
- Jimmy Song - Programming Bitcoin Chapters 13

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core (v22 or higher) with the application data is stored in 
```$HOME/Library/Application Support/Bitcoin```.

### Setup bitcoind in regtest
Start up regtest mode, delete any regtest network history so we are starting from scratch. Create a wallet so that we can fund our first output using the `sendtoaddress` command. Mine 101 blocks so that the mining reward first block will have sufficient block maturity (100 blocks) to spend from.

In [2]:
# Make sure bitcoind is not already running
os.system("bitcoin-cli -regtest stop")
time.sleep(2) 

# Delete any previous files to restart regtest
os.system("rm -rfv $HOME/Library/Application\ Support/Bitcoin/regtest/")

# Start up bitcoind in regtest mode
os.system("bitcoind -regtest -daemon -fallbackfee=0.0002")
time.sleep(1.5)

# Create a new wallet and address that we can mine blocks so that we can fund our transactions
wallet = subprocess.getoutput("bitcoin-cli -regtest createwallet mywallet")
address = subprocess.getoutput("bitcoin-cli -regtest getnewaddress")

# Generate 101 blocks so that the first block's block reward reaches maturity
result = subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 101 {addr}".format(addr=address))

# Check that we were able to mine 101 blocks
blockcount = subprocess.getoutput("bitcoin-cli -regtest getblockcount")
assert(blockcount == "101")

### Create a P2WPKH UTXO

In order to create a transaction spending from a P2WPKH UTXO, we'll first need to create the UTXO that is locked with a p2wpkh script. To do that, we'll create a P2WPKH address from a private key, and fund it using the bitcoind wallet created in the setup step.

#### Create a p2wpkh address 
For more on this step, review the 'Addresses' notebook.

In [3]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
sender_p2wpkh_addr = pk_to_p2wpkh(sender_pubkey, network = "regtest")
print("sender's p2wpkh address: " + sender_p2wpkh_addr)

sender's p2wpkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

In [4]:
txid_to_spend = subprocess.getoutput("bitcoin-cli -regtest sendtoaddress " + sender_p2wpkh_addr + " 2.001")
print(txid_to_spend)

a2316c586bbef7192d6774fa4338fef904ead128e9ec510c886d1acd42adede0


We can view the transaction using the bitcoin-cli commands `getrawtransaction` and `decoderawtransaction` as follows:

In [5]:
raw_tx = subprocess.getoutput("bitcoin-cli -regtest getrawtransaction " + txid_to_spend)
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + raw_tx)
# print(decoded)

#### Find which output index the btc was sent to
Since we only sent 2.001 btc of the coinbase transaction (50 btc) to our address, bitcoind creates a change output to send the rest of the btc. By looking at the outputs we can see which is the change output and which was sent to our address. To do this in python we can do the following:

In [6]:
d = json.loads(decoded)
if d["vout"][0]["scriptPubKey"]["address"] == sender_p2wpkh_addr:
    index_to_spend = 0
elif d["vout"][1]["scriptPubKey"]["address"] == sender_p2wpkh_addr:
    index_to_spend = 1
else:
    raise Exception("couldn't find output")
print("index to spend from: " + str(index_to_spend))

index to spend from: 1


#### Mine a block so that the funding tx gets confirmed

In [7]:
subprocess.getoutput("bitcoin-cli -regtest generatetoaddress 1 {addr}".format(addr=address));

## Spending a p2wpkh UTXO

Now that we have some funds locked up in a p2wpkh utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`.

### Decoding a bech32 address

The first thing we need to do is decode the address by doing this we can achieve the following:
1 - validate the checksum to know the address was transmitted without error
2 - make sure we are sending btc on the correct network (testnet/mainnet)
3 - know what to put in the scriptPubkey

For more on addresses, refer back to the 'Addresses' chapter.

In [8]:
receiver_address = 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw'

# human readable part
hrp = 'bcrt'
witver, witprog = bech32.decode(hrp, receiver_address)

pubkey_hash = bytearray(witprog)
print("witness version: ", witver)
print("pubkey/script hash: ", pubkey_hash.hex())

witness version:  0
pubkey/script hash:  fc7250a211deddc70ee5a2738de5f07817351cef


The witness version `0`, tells us that this address corresponds to either a p2wpkh or p2wsh output. Then, we can tell from the length of the hash whether it is a pubkey hash (20 bytes long), or a script hash (32 bytes long). 

For more on decoding addresses, refer back to the 'Addresses' chapter.

Now we can create the receiver's output scriptPubkey:

In [9]:
receiver_spk = bytes.fromhex("0014") + pubkey_hash

### Create an unsigned p2wpkh transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [10]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `sender_privkey` and `sender_pubkey`

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("0014") + hash160(change_pubkey)

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [11]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  0200000001e0edad42cd1a6d880c51ece928d1ea04f9fe3843fa74672d19f7be6b586c31a20100000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [12]:
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + unsigned_tx.hex())
# print(decoded)

Segwit transactions have a new signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)


In [13]:
pk_hash = hash160(sender_pubkey)
scriptcode = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

input_amount_sat = int(2.001 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

02000000ca281ce620db9058ee68692fa39328e3aac050838e1c8c39d84a6f9e9a03b4e63bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044e0edad42cd1a6d880c51ece928d1ea04f9fe3843fa74672d19f7be6b586c31a2010000001976a914fc7250a211deddc70ee5a2738de5f07817351cef88aca048ed0b00000000fffffffff91a6606c3037951dc241bb6edf31f9a61112940431aee9593c16c3bb3e1d2600000000001000000


Now we are ready to hash this transaction and produce an ecdsa signature on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [14]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items for the txin
    # 2 items for signature and pubkey
    bytes.fromhex("02")
    + pushbytes(signature)
    + pushbytes(sender_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  02000000000101e0edad42cd1a6d880c51ece928d1ea04f9fe3843fa74672d19f7be6b586c31a20100000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d02483045022100814acf90727bc137c2916d8c014ec38b36e9aa4ecdfdadd2a9e0b823b7f77a3402200f6d4e074ffded77a0d58f55243aab4adfdf362c40f81ddaf297027dac426daf0121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa00000000


### Broadcase the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [15]:
new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + signed_tx.hex())
# new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_tx.hex()+ "\"]'")

print(new_tx_txid)

e275e4e72704721a79082ebbeaa9707b2cda249a43f8bf9be478eb62c1507bc1


We can decode the serialized transaction using ```decoderawtransaction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [16]:
print("receiver's p2pkh address: " + receiver_address)
change_p2pkh_addr = pk_to_p2pkh(change_pubkey, network = "regtest")
print("sender's change p2pkh address: " + change_p2pkh_addr)

receiver's p2pkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw
sender's change p2pkh address: mo6CPsdW8EsnWdmSSCrQ6225VVDtpMBTug


In [17]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx.hex())
print(decoded)

{
  "txid": "e275e4e72704721a79082ebbeaa9707b2cda249a43f8bf9be478eb62c1507bc1",
  "hash": "a0883dacb72edefb22b20a2aed247b5ab9613d4ca556f48710a8b4ce5ca09bdb",
  "version": 2,
  "size": 223,
  "vsize": 141,
  "weight": 562,
  "locktime": 0,
  "vin": [
    {
      "txid": "a2316c586bbef7192d6774fa4338fef904ead128e9ec510c886d1acd42adede0",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3045022100814acf90727bc137c2916d8c014ec38b36e9aa4ecdfdadd2a9e0b823b7f77a3402200f6d4e074ffded77a0d58f55243aab4adfdf362c40f81ddaf297027dac426daf01",
        "034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 1.50000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f0

## Quiz


 ## Answers
    

## Exercise
